In [1]:
"""
Attention采样第一阶段做法，直接使用矩阵相乘，不考虑权重及Attention。
"""
import numpy as np
import networkx as nx
import random
import torch
from train import Train_Model
#from tqdm import tqdm, trange
from tqdm import tqdm_notebook as tqdm
import gc
from model import *
from utils import *
print(torch.__version__)
print(torch.cuda.is_available())

1.7.0
True


In [2]:
def pre_sample_for_ether(walk_times,adj_sparse, train_index, batch_size, save_name, do_walk, features=None):
    if do_walk:
        nodes_num = len(train_index)
        walks = torch.zeros(nodes_num, walk_times+1).long().cuda()
        batches = create_batches_forList(train_index, batch_size, True)
        i=0
        candi_node = 0
        indexes = np.arange(0, adj_sparse.shape[0])
        for batch in batches:
            walks[i*batch_size : i*batch_size + len(batch), 0] = torch.tensor(batch).cuda()
            node_adj = adj_sparse[batch.cpu().numpy()]
            candi_node = sparse_mx_to_torch_sparse_tensor(node_adj).cuda()
            chosen_node = torch.zeros(len(batch), adj_sparse.shape[0]).cuda()
            for id in range(len(batch)):
                chosen_node[id][batch[id]] = 1.
            candi_node = ((- chosen_node + candi_node)> 0.0).float()
            for x in range(candi_node.shape[0]):
                if candi_node[x].sum() < 0:
                    print("x = {}".format(x))
                if candi_node[x].sum()==0:
                    candi_node[x][batch[x]] = 1.
                else:
                    x_feat = features[batch[x]]
                    x_feat = torch.reshape(x_feat, shape=(x_feat.shape[0], 1))
                    batch_x_list = np.array([batch[x]], dtype=np.int32)
                    tmp_adj = adj_sparse[batch_x_list]
                    tmp_adj = np.array(tmp_adj.toarray()[0], dtype=np.bool)
                    tmp_adj = indexes[tmp_adj]
                    for y in tmp_adj:
                        y_feat = features[y]
                        y_feat = torch.reshape(y_feat, shape=(1, y_feat.shape[0]))
                        mm_value = torch.mm(y_feat, x_feat)[0][0]
                        if mm_value >= 0:
                            candi_node[x][y] = mm_value
                        else:
                            candi_node[x][y] = 0.

                            
            for walk_id in range(walk_times):
                p = candi_node
                new_node = torch.multinomial(p,1).squeeze(1)
                walks[i*batch_size : i*batch_size + len(batch), walk_id+1] = new_node
                for id in range(len(batch)):
                    chosen_node[id][new_node[id]] = 1.
                candi_node = candi_node - chosen_node + sparse_mx_to_torch_sparse_tensor(adj_sparse[new_node.cpu()]).cuda()
                candi_node = (candi_node> 0.0).float()

            i+=1
        torch.cuda.empty_cache()
        result1 = np.array(walks.cpu())
        io.savemat(save_name+'.mat',{save_name:result1})
        return walks
    else:
        walks = io.loadmat(save_name+'.mat')
        print("walks",walks)
        return torch.tensor(walks[save_name]).cuda()

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
sparse_adj, sparse_adj_train,sparse_adj_train_all,features, train_feature,train_feature_all,labels, train_labels, id_train, id_valid, id_test, num_labels = Origin_load_ether_data()
sparse_adj.setdiag(1.0) 
sparse_adj_train.setdiag(1.0) 
sparse_adj_train_all.setdiag(1.0) 

features = (1402220, 12) type = <class 'numpy.ndarray'>


/home/ubuntu/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [4]:
for walk in range(1,20):
    for seed in range(10,20):
        torch.manual_seed(seed)
        random.seed(seed)
        torch.cuda.manual_seed(seed)
        np.random.seed(seed)
        save_name = 'seed='+str(seed)+'_walk='+str(walk)
        print('process...'+'train_'+save_name)
        pre_sample_for_ether(walk,sparse_adj_train_all, id_train, 32, './walks_ether_attention_feat_matrix/train_'+save_name, True, features=train_feature_all)
        print('process...'+'valid_'+save_name)
        pre_sample_for_ether(walk,sparse_adj, id_valid, 32, './walks_ether_attention_feat_matrix/valid_'+save_name, True, features=train_feature_all)
        print('process...'+'test_'+save_name)
        pre_sample_for_ether(walk,sparse_adj, id_test, 32, './walks_ether_attention_feat_matrix/test_'+save_name, True, features=train_feature_all)

process...train_seed=10_walk=1


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


process...valid_seed=10_walk=1
process...test_seed=10_walk=1
process...train_seed=11_walk=1
process...valid_seed=11_walk=1
process...test_seed=11_walk=1
process...train_seed=12_walk=1
process...valid_seed=12_walk=1
process...test_seed=12_walk=1
process...train_seed=13_walk=1


KeyboardInterrupt: 